In [1]:
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver

## 데이터 먼저 확인

In [2]:
data = pd.read_csv('data/qwer_stats.csv')
data

,name,callName,dateOfBirth,tournament,team,championId,games,wins,winRate,kills,...,cs15,gold15,ks,base,gd15,csd15,xpd15,lvd15,loses,kda
0,신정현,Nuclear,1997-05-18T15:00:00.000Z,LCK 2020 Spring Season,DWG KIA,145,5,1,0.2000,2.20,...,131,5155,0.3219,9.20,-686,-14.6,-468,-0.4,4,3.25
1,신정현,Nuclear,1997-05-18T15:00:00.000Z,LCK 2020 Spring Season,DWG KIA,21,4,4,1.0000,3.75,...,131,5349,0.2632,11.25,116,-7.3,26,0.0,0,14.66
2,신정현,Nuclear,1997-05-18T15:00:00.000Z,LCK 2020 Spring Season,DWG KIA,523,9,3,0.3333,3.22,...,132,5091,0.2776,11.44,-347,1.8,-201,0.1,6,4.07
3,신정현,Nuclear,1997-05-18T15:00:00.000Z,LCK 2020 Spring Season,DWG KIA,498,3,2,0.6666,3.00,...,141,5387,0.3500,12.66,68,4.3,69,0.0,1,2.50
4,신정현,Nuclear,1997-05-18T15:00:00.000Z,LCK 2020 Spring Season,DWG KIA,235,1,1,1.0000,1.00,...,17,3490,0.0833,11.00,-1687,-120.0,-805,-1.0,0,10.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65033,Hugo Dias de Faria,Hugato,NaN,CBLOL 2021 Split 2 Season,Netshoes Miners,5,1,0,0.0000,1.00,...,98,5222,0.1000,11.00,-41,-15.0,-509,0.0,1,3.00
65034,Hugo Dias de Faria,Hugato,NaN,CBLOL 2021 Split 2 Season,Netshoes Miners,131,1,0,0.0000,0.00,...,102,5201,0.0000,8.00,913,12.0,397,1.0,1,0.60
65035,Okan Öztopaç,Nako,NaN,TCL 2021 Summer Season,Team AURORA,18,1,0,0.0000,2.00,...,108,4240,0.3333,8.00,-498,94.0,236,0.0,1,1.00
65036,Okan Öztopaç,Nako,NaN,TCL 2021 Summer Season,Team AURORA,523,1,0,0.0000,2.00,...,133,5564,0.2222,9.00,-452,-5.0,-1451,-1.0,1,2.00


In [3]:
# 숫자로 된 각종 Key값에 대한 Value 가져오기
champion_constant = requests.get("http://ddragon.leagueoflegends.com/cdn/11.16.1/data/ko_KR/champion.json")
# Json 파일을 DataFrame으로 변환
champion_df = pd.DataFrame(champion_constant.json()['data']).T[['key','name']]
# 변수형 문자 -> 숫자
champion_df['key'] = pd.to_numeric(champion_df['key'])

In [4]:
# 매칭되는 곳으로 내용 추가하여 새로운 DataFrame으로 저장
qwer_data = pd.merge(data,champion_df,how = 'left', left_on = 'championId', right_on = 'key')
# 중복되는 값 삭제
qwer_data = qwer_data.rename(columns={'name_y': 'Champion'}).drop(['championId','key'],axis = 1)

In [5]:
qwer_data

,name_x,callName,dateOfBirth,tournament,team,games,wins,winRate,kills,deaths,...,gold15,ks,base,gd15,csd15,xpd15,lvd15,loses,kda,Champion
0,신정현,Nuclear,1997-05-18T15:00:00.000Z,LCK 2020 Spring Season,DWG KIA,5,1,0.2000,2.20,1.60,...,5155,0.3219,9.20,-686,-14.6,-468,-0.4,4,3.25,카이사
1,신정현,Nuclear,1997-05-18T15:00:00.000Z,LCK 2020 Spring Season,DWG KIA,4,4,1.0000,3.75,0.75,...,5349,0.2632,11.25,116,-7.3,26,0.0,0,14.66,미스 포츈
2,신정현,Nuclear,1997-05-18T15:00:00.000Z,LCK 2020 Spring Season,DWG KIA,9,3,0.3333,3.22,1.55,...,5091,0.2776,11.44,-347,1.8,-201,0.1,6,4.07,아펠리오스
3,신정현,Nuclear,1997-05-18T15:00:00.000Z,LCK 2020 Spring Season,DWG KIA,3,2,0.6666,3.00,2.00,...,5387,0.3500,12.66,68,4.3,69,0.0,1,2.50,자야
4,신정현,Nuclear,1997-05-18T15:00:00.000Z,LCK 2020 Spring Season,DWG KIA,1,1,1.0000,1.00,1.00,...,3490,0.0833,11.00,-1687,-120.0,-805,-1.0,0,10.00,세나
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65033,Hugo Dias de Faria,Hugato,NaN,CBLOL 2021 Split 2 Season,Netshoes Miners,1,0,0.0000,1.00,2.00,...,5222,0.1000,11.00,-41,-15.0,-509,0.0,1,3.00,신 짜오
65034,Hugo Dias de Faria,Hugato,NaN,CBLOL 2021 Split 2 Season,Netshoes Miners,1,0,0.0000,0.00,5.00,...,5201,0.0000,8.00,913,12.0,397,1.0,1,0.60,다이애나
65035,Okan Öztopaç,Nako,NaN,TCL 2021 Summer Season,Team AURORA,1,0,0.0000,2.00,4.00,...,4240,0.3333,8.00,-498,94.0,236,0.0,1,1.00,트리스타나
65036,Okan Öztopaç,Nako,NaN,TCL 2021 Summer Season,Team AURORA,1,0,0.0000,2.00,1.00,...,5564,0.2222,9.00,-452,-5.0,-1451,-1.0,1,2.00,아펠리오스


In [6]:
# 칼럼의 수와 칼럼들 확인
qwer_data.columns

Index(['name_x', 'callName', 'dateOfBirth', 'tournament', 'team', 'games',
       'wins', 'winRate', 'kills', 'deaths', 'assists', 'visionScore', 'wpm',
       'wkm', 'dmg', 'dpm', 'dtpm', 'kapm', 'gold', 'gpm', 'cs', 'cspm',
       'counterJungle', 'kpar', 'dpar', 'xp15', 'cs15', 'gold15', 'ks', 'base',
       'gd15', 'csd15', 'xpd15', 'lvd15', 'loses', 'kda', 'Champion'],
      dtype='object')

In [7]:
# 생일과 이름은 불필요하니 제거함
qwer_data2= qwer_data.drop(columns=['dateOfBirth',"name_x"])
qwer_data2

,callName,tournament,team,games,wins,winRate,kills,deaths,assists,visionScore,...,gold15,ks,base,gd15,csd15,xpd15,lvd15,loses,kda,Champion
0,Nuclear,LCK 2020 Spring Season,DWG KIA,5,1,0.2000,2.20,1.60,3.00,29.60,...,5155,0.3219,9.20,-686,-14.6,-468,-0.4,4,3.25,카이사
1,Nuclear,LCK 2020 Spring Season,DWG KIA,4,4,1.0000,3.75,0.75,7.25,46.75,...,5349,0.2632,11.25,116,-7.3,26,0.0,0,14.66,미스 포츈
2,Nuclear,LCK 2020 Spring Season,DWG KIA,9,3,0.3333,3.22,1.55,3.11,36.77,...,5091,0.2776,11.44,-347,1.8,-201,0.1,6,4.07,아펠리오스
3,Nuclear,LCK 2020 Spring Season,DWG KIA,3,2,0.6666,3.00,2.00,2.00,46.66,...,5387,0.3500,12.66,68,4.3,69,0.0,1,2.50,자야
4,Nuclear,LCK 2020 Spring Season,DWG KIA,1,1,1.0000,1.00,1.00,9.00,115.00,...,3490,0.0833,11.00,-1687,-120.0,-805,-1.0,0,10.00,세나
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65033,Hugato,CBLOL 2021 Split 2 Season,Netshoes Miners,1,0,0.0000,1.00,2.00,5.00,38.00,...,5222,0.1000,11.00,-41,-15.0,-509,0.0,1,3.00,신 짜오
65034,Hugato,CBLOL 2021 Split 2 Season,Netshoes Miners,1,0,0.0000,0.00,5.00,3.00,21.00,...,5201,0.0000,8.00,913,12.0,397,1.0,1,0.60,다이애나
65035,Nako,TCL 2021 Summer Season,Team AURORA,1,0,0.0000,2.00,4.00,2.00,24.00,...,4240,0.3333,8.00,-498,94.0,236,0.0,1,1.00,트리스타나
65036,Nako,TCL 2021 Summer Season,Team AURORA,1,0,0.0000,2.00,1.00,0.00,24.00,...,5564,0.2222,9.00,-452,-5.0,-1451,-1.0,1,2.00,아펠리오스


In [8]:
# 스프링 시즌만 확인하기 위해 LCK 스프링만 확인
qwer_data3 = qwer_data2[qwer_data2['tournament'] == 'LCK 2021 Spring Season']
qwer_data3

,callName,tournament,team,games,wins,winRate,kills,deaths,assists,visionScore,...,gold15,ks,base,gd15,csd15,xpd15,lvd15,loses,kda,Champion
140,Flawless,LCK 2021 Spring Season,Gen.G,1,1,1.0000,2.00,3.00,12.00,52.00,...,4651,0.0869,12.00,-324,-1.0,-422,0.0,0,4.66,판테온
141,Flawless,LCK 2021 Spring Season,Gen.G,2,1,0.5000,4.00,2.00,3.00,44.00,...,4875,0.4444,9.50,44,4.0,562,1.0,1,3.50,올라프
142,Flawless,LCK 2021 Spring Season,Gen.G,1,1,1.0000,2.00,1.00,5.00,91.00,...,5148,0.1428,19.00,-58,-21.0,-1002,-1.0,0,7.00,릴리아
143,Flawless,LCK 2021 Spring Season,Gen.G,1,0,0.0000,0.00,0.00,0.00,69.00,...,4500,0.0000,11.00,-1828,-26.0,-1825,-1.0,1,0.00,우디르
1662,Bdd,LCK 2021 Spring Season,Gen.G,14,10,0.7142,2.35,1.78,6.14,42.57,...,5491,0.1646,11.57,119,8.2,357,0.2,4,4.76,아지르
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12071,Gumayusi,LCK 2021 Spring Season,T1,3,2,0.6666,4.33,2.33,4.33,52.66,...,5481,0.3493,12.33,-520,-7.4,-224,-0.4,1,3.71,베인
12072,Gumayusi,LCK 2021 Spring Season,T1,4,2,0.5000,7.75,2.25,6.00,55.75,...,6214,0.4928,12.25,1305,24.5,615,0.5,2,6.11,사미라
12073,Gumayusi,LCK 2021 Spring Season,T1,6,2,0.3333,2.50,2.00,2.66,46.33,...,5420,0.3326,12.50,-109,5.1,172,0.1,4,2.58,카이사
12074,Gumayusi,LCK 2021 Spring Season,T1,2,0,0.0000,3.50,2.50,1.00,29.00,...,5206,0.4375,10.50,-231,1.5,-219,0.0,2,1.80,진


In [14]:
# 고스트 선수의 데이터만 확인해봄
a= qwer_data3[(qwer_data3['callName']=='Ghost') ]

In [15]:
# 보기 쉬운 형태로 바꿔주기 위해 필요 컬럼만 뽑아주고 정렬
b = a[['Champion','games','wins','loses','winRate','kills','deaths','assists','kda','cs','cspm','gold','gpm','kpar','ks']].sort_values('games', ascending =False)

In [17]:
# 데이터를 미리 저장해주기
b.to_csv('Ghost.csv', encoding="CP949")

In [41]:
# 룰러 선수 데이터를 확인
Ruler_data = qwer_data3[qwer_data3['callName']== 'Ruler']
Ruler_data[['gd15','csd15','xpd15']].mean()

gd15     370.000000
csd15      8.118182
xpd15     95.909091
dtype: float64

In [44]:
Ruler_data.columns

Index(['callName', 'tournament', 'team', 'games', 'wins', 'winRate', 'kills',
       'deaths', 'assists', 'visionScore', 'wpm', 'wkm', 'dmg', 'dpm', 'dtpm',
       'kapm', 'gold', 'gpm', 'cs', 'cspm', 'counterJungle', 'kpar', 'dpar',
       'xp15', 'cs15', 'gold15', 'ks', 'base', 'gd15', 'csd15', 'xpd15',
       'lvd15', 'loses', 'kda', 'Champion'],
      dtype='object')

In [50]:
Ruler_data

,callName,tournament,team,games,wins,winRate,kills,deaths,assists,visionScore,...,gold15,ks,base,gd15,csd15,xpd15,lvd15,loses,kda,Champion
6068,Ruler,LCK 2021 Spring Season,Gen.G,7,5,0.7142,3.14,2.00,4.42,50.28,...,5874,0.1646,10.71,798,35.5,692,0.7,2,3.78,아펠리오스
6069,Ruler,LCK 2021 Spring Season,Gen.G,5,5,1.0000,4.40,0.60,6.60,81.60,...,5990,0.2985,10.20,1025,39.2,593,0.8,0,18.33,칼리스타
6070,Ruler,LCK 2021 Spring Season,Gen.G,3,2,0.6666,3.66,2.00,7.00,40.00,...,5810,0.1812,12.33,240,4.3,408,0.6,1,5.33,진
6071,Ruler,LCK 2021 Spring Season,Gen.G,1,1,1.0000,5.00,2.00,11.00,153.00,...,4233,0.2631,16.00,-558,-111.0,-1171,-1.0,0,8.00,세나
6072,Ruler,LCK 2021 Spring Season,Gen.G,6,4,0.6666,2.83,1.83,4.33,56.83,...,5301,0.1730,11.83,185,5.0,303,0.5,2,3.90,자야
6073,Ruler,LCK 2021 Spring Season,Gen.G,2,1,0.5000,2.00,2.50,2.00,38.50,...,5339,0.2588,10.00,327,54.0,163,0.5,1,1.60,이즈리얼
6074,Ruler,LCK 2021 Spring Season,Gen.G,4,1,0.2500,4.25,2.25,3.00,89.50,...,5445,0.2457,12.00,661,25.5,286,0.5,3,3.22,트리스타나
6075,Ruler,LCK 2021 Spring Season,Gen.G,16,9,0.5625,3.87,1.50,2.56,66.56,...,5588,0.3448,11.18,127,11.6,89,-0.2,7,4.29,카이사
6076,Ruler,LCK 2021 Spring Season,Gen.G,4,4,1.0000,5.50,0.25,4.50,70.25,...,6053,0.4383,11.00,1024,22.2,578,0.7,0,40.00,징크스
6077,Ruler,LCK 2021 Spring Season,Gen.G,1,0,0.0000,3.00,2.00,1.00,32.00,...,5337,0.3750,10.00,253,8.0,102,0.0,1,2.00,베인


In [54]:
# 게임 수가 한판인 게임은 실험적 픽이라고 고려하여 제외함
Ruler_data2 =Ruler_data[Ruler_data['games'] != 1][['callName','Champion','games','wins','loses','winRate','kda','dmg','dpm','kpar','dpar']]
Ruler_data2

# 룰러 선수는 세나를 사용하지 않는다.
# 왜 룰러선수는 세나를 사용하지 않을까

,callName,Champion,games,wins,loses,winRate,kda,dmg,dpm,kpar,dpar
6068,Ruler,아펠리오스,7,5,2,0.7142,3.78,16037,491,0.4966,0.2508
6069,Ruler,칼리스타,5,5,0,1.0000,18.33,14374,473,0.7408,0.2607
6070,Ruler,진,3,2,1,0.6666,5.33,10552,336,0.6294,0.1896
6072,Ruler,자야,6,4,2,0.6666,3.90,15532,463,0.4705,0.2255
6073,Ruler,이즈리얼,2,1,1,0.5000,1.60,20541,663,0.4470,0.3135
6074,Ruler,트리스타나,4,1,3,0.2500,3.22,18200,468,0.4476,0.2815
6075,Ruler,카이사,16,9,7,0.5625,4.29,12620,401,0.5955,0.2532
6076,Ruler,징크스,4,4,0,1.0000,40.00,25461,745,0.8339,0.4177


In [75]:
# 룰러의 스탯의 평균을 확인
a = Ruler_data2[['kpar']].mean()
b = Ruler_data2[['dpar']].mean()

print(f'Ruler 선수의 킬 관여율은 평균 {a.values.round(2)} 이다')
print(f'Ruler 선수의 데미지 비중은 평균 {b.values.round(2)} 이다')

Ruler 선수의 킬 관여율은 평균 [0.58] 이다
Ruler 선수의 데미지 비중은 평균 [0.27] 이다


## 고스트 선수 스탯

In [76]:
Ghost_data = qwer_data3[qwer_data3['callName']== 'Ghost']
Ghost_data

,callName,tournament,team,games,wins,winRate,kills,deaths,assists,visionScore,...,gold15,ks,base,gd15,csd15,xpd15,lvd15,loses,kda,Champion
5936,Ghost,LCK 2021 Spring Season,DWG KIA,7,7,1.0000,4.57,1.00,9.28,84.85,...,4515,0.2517,10.71,-435,-110.3,-911,-0.8,0,13.85,세나
5937,Ghost,LCK 2021 Spring Season,DWG KIA,3,3,1.0000,4.33,1.00,4.66,42.33,...,5512,0.2996,9.00,243,-1.7,311,0.3,0,9.00,트리스타나
5938,Ghost,LCK 2021 Spring Season,DWG KIA,20,16,0.8000,5.05,1.65,4.75,60.40,...,5309,0.3163,11.80,-184,-6.0,-162,-0.2,4,5.93,카이사
5939,Ghost,LCK 2021 Spring Season,DWG KIA,1,1,1.0000,4.00,1.00,10.00,38.00,...,5082,0.2105,12.00,-1161,-12.0,-341,0.0,0,14.00,바루스
5940,Ghost,LCK 2021 Spring Season,DWG KIA,1,1,1.0000,8.00,0.00,4.00,51.00,...,5325,0.6153,10.00,118,8.0,-326,-1.0,0,14.40,징크스
5941,Ghost,LCK 2021 Spring Season,DWG KIA,3,2,0.6666,5.33,2.33,5.33,72.66,...,5730,0.2970,13.33,410,9.0,530,0.0,1,4.57,칼리스타
5942,Ghost,LCK 2021 Spring Season,DWG KIA,3,1,0.3333,1.66,2.00,3.00,46.00,...,5501,0.2262,13.00,94,-0.7,-139,0.0,2,2.33,아펠리오스
5943,Ghost,LCK 2021 Spring Season,DWG KIA,3,2,0.6666,4.66,2.66,4.00,44.66,...,5083,0.3892,10.00,-485,28.0,220,0.0,1,3.25,이즈리얼
5944,Ghost,LCK 2021 Spring Season,DWG KIA,7,6,0.8571,5.42,2.28,5.57,41.71,...,5395,0.3271,11.42,150,-8.5,157,0.1,1,4.81,자야
5945,Ghost,LCK 2021 Spring Season,DWG KIA,1,0,0.0000,0.00,2.00,1.00,41.00,...,5311,0.0000,13.00,-216,-12.0,-164,0.0,1,0.50,사미라


In [77]:
# 마찬가지로 1번만 픽된 챔피언은 제거해주고 필요한 스탯만 가져오기
Ghost_data2 =Ghost_data[Ghost_data['games'] != 1][['callName','Champion','games','wins','loses','winRate','kda','dmg','dpm','kpar','dpar']]
Ghost_data2

,callName,Champion,games,wins,loses,winRate,kda,dmg,dpm,kpar,dpar
5936,Ghost,세나,7,7,0,1.0000,13.85,17274,572,0.7777,0.2295
5937,Ghost,트리스타나,3,3,0,1.0000,9.00,13523,468,0.6414,0.2346
5938,Ghost,카이사,20,16,4,0.8000,5.93,16662,476,0.6250,0.2173
5941,Ghost,칼리스타,3,2,1,0.6666,4.57,15178,417,0.6239,0.1700
5942,Ghost,아펠리오스,3,1,2,0.3333,2.33,12718,354,0.6032,0.2130
5943,Ghost,이즈리얼,3,2,1,0.6666,3.25,17781,576,0.7393,0.2637
5944,Ghost,자야,7,6,1,0.8571,4.81,18184,525,0.6474,0.2409


In [80]:
# 두 선수의 영향력을 확인하기 위해 킬관여율과 데미지 비중을 확인함

a = Ruler_data2[['kpar']].mean()
b = Ruler_data2[['dpar']].mean()
c = Ghost_data2[['kpar']].mean()
d = Ghost_data2[['dpar']].mean()



print(f'Ruler 선수의 킬 관여율은 평균 {a.values.round(2)} 이다')
print(f'Ruler 선수의 데미지 비중은 평균 {b.values.round(2)} 이다')
print('---'* 10)
print(f'Ghost 선수의 킬 관여율은 평균 {c.values.round(2)} 이다')
print(f'Ghost 선수의 데미지 비중은 평균 {d.values.round(2)} 이다')

Ruler 선수의 킬 관여율은 평균 [0.58] 이다
Ruler 선수의 데미지 비중은 평균 [0.27] 이다
------------------------------
Ghost 선수의 킬 관여율은 평균 [0.67] 이다
Ghost 선수의 데미지 비중은 평균 [0.22] 이다


## 모든 원딜들의 스탯 저장

In [96]:
Gumayusi_data = qwer_data3[qwer_data3['callName'] == 'Gumayusi']
Teddy_data = qwer_data3[qwer_data3['callName'] == 'Teddy']
Bang_data = qwer_data3[qwer_data3['callName'] == 'Bang']
Ghost_data = qwer_data3[qwer_data3['callName'] == 'Ghost']
Ruler_data = qwer_data3[qwer_data3['callName'] == 'Ruler']
Deft_data = qwer_data3[qwer_data3['callName'] == 'Deft']

BAO_data = qwer_data3[qwer_data3['callName'] == 'BAO']
Leo_data = qwer_data3[qwer_data3['callName'] == 'Leo']
Noah_data = qwer_data3[qwer_data3['callName'] == 'Noah']
Prince_data = qwer_data3[qwer_data3['callName'] == 'Prince']
deokdam_data = qwer_data3[qwer_data3['callName'] == 'deokdam']
Hena_data = qwer_data3[qwer_data3['callName'] == 'Hena']

In [105]:
# 활용하기 편하게 콘캣으로 모든 원딜들의 데이터를 합침

ad_carry = pd.concat([Gumayusi_data,
Teddy_data,
Bang_data,
Ghost_data,
Ruler_data,
Deft_data,
BAO_data,
Leo_data,
Noah_data,
Prince_data,
deokdam_data,
Hena_data],
         axis=0)

In [110]:
ad_carry

,callName,tournament,team,games,wins,winRate,kills,deaths,assists,visionScore,...,gold15,ks,base,gd15,csd15,xpd15,lvd15,loses,kda,Champion
12070,Gumayusi,LCK 2021 Spring Season,T1,5,4,0.8000,5.20,1.00,3.40,48.40,...,6037,0.3790,11.80,774,15.8,217,0.0,1,8.60,아펠리오스
12071,Gumayusi,LCK 2021 Spring Season,T1,3,2,0.6666,4.33,2.33,4.33,52.66,...,5481,0.3493,12.33,-520,-7.4,-224,-0.4,1,3.71,베인
12072,Gumayusi,LCK 2021 Spring Season,T1,4,2,0.5000,7.75,2.25,6.00,55.75,...,6214,0.4928,12.25,1305,24.5,615,0.5,2,6.11,사미라
12073,Gumayusi,LCK 2021 Spring Season,T1,6,2,0.3333,2.50,2.00,2.66,46.33,...,5420,0.3326,12.50,-109,5.1,172,0.1,4,2.58,카이사
12074,Gumayusi,LCK 2021 Spring Season,T1,2,0,0.0000,3.50,2.50,1.00,29.00,...,5206,0.4375,10.50,-231,1.5,-219,0.0,2,1.80,진
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11596,Hena,LCK 2021 Spring Season,Fredit BRION,5,3,0.6000,3.00,1.20,5.40,55.40,...,5286,0.2798,11.40,-28,-0.4,25,0.0,2,7.00,칼리스타
11597,Hena,LCK 2021 Spring Season,Fredit BRION,2,0,0.0000,1.00,3.00,0.50,38.00,...,4995,0.5000,13.00,-1255,-20.0,-745,0.0,2,0.50,자야
11598,Hena,LCK 2021 Spring Season,Fredit BRION,1,0,0.0000,2.00,3.00,3.00,30.00,...,5105,0.2857,10.00,109,11.0,492,1.0,1,1.66,사미라
11599,Hena,LCK 2021 Spring Season,Fredit BRION,1,0,0.0000,1.00,4.00,7.00,52.00,...,5418,0.1111,16.00,689,5.0,617,0.0,1,2.00,애쉬


In [252]:
# 마찬가지로 한판만 진행된 게임은 제거
ad_carry2 =ad_carry[ad_carry['games'] != 1][['callName','Champion','games','wins','loses','winRate','kda','dmg','dpm','kpar','dpar']]
ad_carry2

,callName,Champion,games,wins,loses,winRate,kda,dmg,dpm,kpar,dpar
12070,Gumayusi,아펠리오스,5,4,1,0.8000,8.60,15119,482,0.6159,0.2821
12071,Gumayusi,베인,3,2,1,0.6666,3.71,13095,399,0.6828,0.2445
12072,Gumayusi,사미라,4,2,2,0.5000,6.11,16286,475,0.8675,0.2677
12073,Gumayusi,카이사,6,2,4,0.3333,2.58,9337,304,0.7201,0.2052
12074,Gumayusi,진,2,0,2,0.0000,1.80,11194,385,0.6375,0.2517
...,...,...,...,...,...,...,...,...,...,...,...
11594,Hena,트리스타나,9,4,5,0.4444,4.56,12879,409,0.8094,0.2564
11595,Hena,카이사,12,4,8,0.3333,3.43,12883,366,0.7563,0.2138
11596,Hena,칼리스타,5,3,2,0.6000,7.00,11356,335,0.7326,0.2118
11597,Hena,자야,2,0,2,0.0000,0.50,14536,434,1.0000,0.3085


In [159]:
# 룰러가 플레이한 챔피언의 정보를 확인
Ruler_data2.sort_values(by='games', ascending=False)

,callName,Champion,games,wins,loses,winRate,kda,dmg,dpm,kpar,dpar
6075,Ruler,카이사,16,9,7,0.5625,4.29,12620,401,0.5955,0.2532
6068,Ruler,아펠리오스,7,5,2,0.7142,3.78,16037,491,0.4966,0.2508
6072,Ruler,자야,6,4,2,0.6666,3.90,15532,463,0.4705,0.2255
6069,Ruler,칼리스타,5,5,0,1.0000,18.33,14374,473,0.7408,0.2607
6074,Ruler,트리스타나,4,1,3,0.2500,3.22,18200,468,0.4476,0.2815
6076,Ruler,징크스,4,4,0,1.0000,40.00,25461,745,0.8339,0.4177
6070,Ruler,진,3,2,1,0.6666,5.33,10552,336,0.6294,0.1896
6073,Ruler,이즈리얼,2,1,1,0.5000,1.60,20541,663,0.4470,0.3135


In [124]:
# 룰러, 고스트와  리그 원딜러들의 평균을 확인

a = Ruler_data2[['kpar']].mean()
b = Ruler_data2[['dpar']].mean()
c = Ghost_data2[['kpar']].mean()
d = Ghost_data2[['dpar']].mean()
e = ad_carry2[['kpar']].mean()
f = ad_carry2[['dpar']].mean()

g = Ruler_data2[['dpm']].mean()
h=  Ghost_data2[['dpm']].mean()
i=  ad_carry2[['dpm']].mean()


print(f'Ruler 선수의 킬 관여율은 평균 {a.values.round(2)} 이다')
print(f'Ruler 선수의 데미지 비중은 평균 {b.values.round(2)} 이다')
print(f'Ruler 선수의 평균 분당 데미지는 {g.values.round(2)} 이다')
print('---'* 10)
print(f'Ghost 선수의 킬 관여율은 평균 {c.values.round(2)} 이다')
print(f'Ghost 선수의 데미지 비중은 평균 {d.values.round(2)} 이다')
print(f'Ghost 선수의 평균 분당 데미지는 {h.values.round(2)} 이다')
print('---'* 10)
print(f'리그 원딜 선수들의 킬 관여율은 평균 {e.values.round(2)} 이다')
print(f'리그 원딜 선수들의 데미지 비중은 평균 {f.values.round(2)} 이다')
print(f'리그 원딜 선수들의 평균 분당 데미지는 {i.values.round(2)} 이다')

Ruler 선수의 킬 관여율은 평균 [0.58] 이다
Ruler 선수의 데미지 비중은 평균 [0.27] 이다
Ruler 선수의 평균 분당 데미지는 [505.] 이다
------------------------------
Ghost 선수의 킬 관여율은 평균 [0.67] 이다
Ghost 선수의 데미지 비중은 평균 [0.22] 이다
Ghost 선수의 평균 분당 데미지는 [484.] 이다
------------------------------
리그 원딜 선수들의 킬 관여율은 평균 [0.62] 이다
리그 원딜 선수들의 데미지 비중은 평균 [0.25] 이다
리그 원딜 선수들의 평균 분당 데미지는 [478.3] 이다


In [121]:
# 승률 100% 의 원딜 픽들

ad_100 = ad_carry[ad_carry['winRate'] == 1]
ad_100_2 = ad_100[ad_100['games'] != 1]
ad_100_2

,callName,tournament,team,games,wins,winRate,kills,deaths,assists,visionScore,...,gold15,ks,base,gd15,csd15,xpd15,lvd15,loses,kda,Champion
6586,Teddy,LCK 2021 Spring Season,T1,2,2,1.0,5.00,1.00,3.50,36.50,...,6026,0.3571,11.00,910,11.5,166,0.0,0,8.50,이즈리얼
6588,Teddy,LCK 2021 Spring Season,T1,2,2,1.0,4.50,3.50,8.00,47.00,...,6610,0.2430,11.00,2117,42.5,780,1.0,0,3.57,칼리스타
5936,Ghost,LCK 2021 Spring Season,DWG KIA,7,7,1.0,4.57,1.00,9.28,84.85,...,4515,0.2517,10.71,-435,-110.3,-911,-0.8,0,13.85,세나
5937,Ghost,LCK 2021 Spring Season,DWG KIA,3,3,1.0,4.33,1.00,4.66,42.33,...,5512,0.2996,9.00,243,-1.7,311,0.3,0,9.00,트리스타나
6069,Ruler,LCK 2021 Spring Season,Gen.G,5,5,1.0,4.40,0.60,6.60,81.60,...,5990,0.2985,10.20,1025,39.2,593,0.8,0,18.33,칼리스타
6076,Ruler,LCK 2021 Spring Season,Gen.G,4,4,1.0,5.50,0.25,4.50,70.25,...,6053,0.4383,11.00,1024,22.2,578,0.7,0,40.00,징크스
6797,Deft,LCK 2021 Spring Season,Hanwha Life Esports,2,2,1.0,4.50,1.00,8.50,40.50,...,5267,0.2302,11.50,-29,9.5,-31,0.0,0,13.00,바루스


In [135]:
## 리그 원딜들의 15분 격차를 통해 라인 주도권 능력에 대해 보려고 함

ad_15mean = ad_carry[['callName','xp15','cs15','gd15','csd15','xpd15']].groupby('callName', as_index=False).mean()
ad_15mean

,callName,xp15,cs15,gd15,csd15,xpd15
0,BAO,5351.000000,131.100000,-438.600000,-7.470000,-287.000000
1,Bang,5660.200000,122.300000,-3.600000,-9.100000,67.100000
2,Deft,5592.333333,133.133333,329.933333,12.366667,104.333333
3,Ghost,5541.600000,124.300000,-146.600000,-10.620000,-82.500000
4,Gumayusi,5773.000000,145.333333,271.500000,9.083333,157.166667
5,Hena,5498.750000,135.000000,-34.500000,0.775000,32.500000
6,Leo,5685.000000,132.400000,-910.800000,-17.280000,-252.000000
7,Noah,5218.400000,112.600000,-44.000000,-17.380000,-228.600000
8,Prince,5568.200000,136.600000,-335.200000,-3.200000,-40.200000
9,Ruler,5596.000000,133.545455,370.000000,8.118182,95.909091


In [149]:
# 평균값 확인 보기 편하게하려고 .T 함
pd.DataFrame( ad_15mean.mean()).T

,xp15,cs15,gd15,csd15,xpd15
0,5526.911806,130.80263,-27.999306,-1.535985,-25.499937


In [256]:
# 리그에서 세나가 원딜로 얼마나 사용되었는지 확인함

ad_carry3  = ad_carry[(ad_carry['Champion'] != '세나') & (ad_carry['games'] != 1)]

In [257]:
ad_carry3

,callName,tournament,team,games,wins,winRate,kills,deaths,assists,visionScore,...,gold15,ks,base,gd15,csd15,xpd15,lvd15,loses,kda,Champion
12070,Gumayusi,LCK 2021 Spring Season,T1,5,4,0.8000,5.20,1.00,3.40,48.40,...,6037,0.3790,11.80,774,15.8,217,0.0,1,8.60,아펠리오스
12071,Gumayusi,LCK 2021 Spring Season,T1,3,2,0.6666,4.33,2.33,4.33,52.66,...,5481,0.3493,12.33,-520,-7.4,-224,-0.4,1,3.71,베인
12072,Gumayusi,LCK 2021 Spring Season,T1,4,2,0.5000,7.75,2.25,6.00,55.75,...,6214,0.4928,12.25,1305,24.5,615,0.5,2,6.11,사미라
12073,Gumayusi,LCK 2021 Spring Season,T1,6,2,0.3333,2.50,2.00,2.66,46.33,...,5420,0.3326,12.50,-109,5.1,172,0.1,4,2.58,카이사
12074,Gumayusi,LCK 2021 Spring Season,T1,2,0,0.0000,3.50,2.50,1.00,29.00,...,5206,0.4375,10.50,-231,1.5,-219,0.0,2,1.80,진
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11594,Hena,LCK 2021 Spring Season,Fredit BRION,9,4,0.4444,3.55,1.77,4.55,61.55,...,5551,0.3082,11.55,121,6.2,107,0.2,5,4.56,트리스타나
11595,Hena,LCK 2021 Spring Season,Fredit BRION,12,4,0.3333,3.50,1.91,3.08,41.50,...,5170,0.3467,11.41,-464,3.0,-60,0.0,8,3.43,카이사
11596,Hena,LCK 2021 Spring Season,Fredit BRION,5,3,0.6000,3.00,1.20,5.40,55.40,...,5286,0.2798,11.40,-28,-0.4,25,0.0,2,7.00,칼리스타
11597,Hena,LCK 2021 Spring Season,Fredit BRION,2,0,0.0000,1.00,3.00,0.50,38.00,...,4995,0.5000,13.00,-1255,-20.0,-745,0.0,2,0.50,자야


In [267]:
## 리그 원딜들의 여러 지표들을 필요한 값들을 확인
ad_15mean3 = ad_carry3[['callName','dmg','dpm','cs','cspm','kpar','dpar','xp15','cs15','gd15','csd15','xpd15']].groupby('callName', as_index=False).mean()
ad_15mean3

,callName,dmg,dpm,cs,cspm,kpar,dpar,xp15,cs15,gd15,csd15,xpd15
0,BAO,15568.000000,440.428571,321.857143,9.398571,0.557114,0.231071,5411.714286,134.571429,-271.285714,-3.242857,-301.428571
1,Bang,14981.571429,459.000000,302.428571,9.201429,0.570871,0.238343,5799.000000,133.285714,357.285714,7.000000,258.857143
2,Deft,18254.777778,554.111111,302.333333,9.264444,0.545578,0.269144,5592.888889,138.555556,185.000000,19.977778,135.555556
3,Ghost,15674.333333,469.333333,315.833333,9.421667,0.646700,0.223250,5622.500000,133.833333,38.000000,3.350000,152.833333
4,Gumayusi,13006.200000,409.000000,321.400000,10.078000,0.704760,0.250240,5715.600000,143.400000,243.800000,7.900000,112.200000
5,Hena,13882.833333,407.166667,341.166667,10.206667,0.800850,0.256567,5501.833333,137.500000,-179.000000,-1.633333,-141.500000
6,Leo,9692.666667,306.666667,286.000000,9.053333,0.534533,0.185633,5631.333333,136.000000,-434.333333,-8.466667,-165.000000
7,Noah,9617.500000,306.000000,287.000000,9.545000,0.368000,0.224850,5324.500000,133.000000,-621.500000,15.300000,-166.500000
8,Prince,15782.333333,473.000000,306.000000,9.593333,0.605400,0.237367,5790.000000,144.333333,-424.000000,2.333333,-66.666667
9,Ruler,16664.625000,505.000000,343.125000,10.450000,0.582662,0.274062,5818.500000,146.125000,548.375000,24.662500,389.000000


In [249]:
# 모든 원딜,  룰러와 고스트를 바로 보기 편하게 정리

display(pd.DataFrame( ad_15mean.mean()).T[['gd15','csd15','xpd15']])

print('Ruler')
display(ad_15mean[ad_15mean['callName']=='Ruler'][['callName','gd15','csd15','xpd15']])
# display(ad_15mean[ad_15mean['callName']=='Ruler'] - ad_15mean.mean())

print('Ghost')
display(ad_15mean[ad_15mean['callName']=='Ghost'][['callName','gd15','csd15','xpd15']])
# display(ad_15mean[ad_15mean['callName']=='Ghost'] - ad_15mean.mean())

,gd15,csd15,xpd15
0,-27.999306,-1.535985,-25.499937


Ruler


,callName,gd15,csd15,xpd15
9,Ruler,370.0,8.118182,95.909091


Ghost


,callName,gd15,csd15,xpd15
3,Ghost,-146.6,-10.62,-82.5


In [276]:
# 다른 지표도 더 넣고 다시 확인

display(pd.DataFrame( ad_15mean3.mean()).T[['dpm','dmg','kpar','dpar','gd15','csd15','xpd15']].round(2))

print('Ruler')
display(ad_15mean3[ad_15mean3['callName']=='Ruler'][['callName','dpm','dmg','kpar','dpar','gd15','csd15','xpd15']].round(2))
# display(ad_15mean[ad_15mean['callName']=='Ruler'] - ad_15mean.mean())

print('Ghost')
display(ad_15mean3[ad_15mean3['callName']=='Ghost'][['callName','dpm','dmg','kpar','dpar','gd15','csd15','xpd15']].round(2))
# display(ad_15mean[ad_15mean['callName']=='Ghost'] - ad_15mean.mean())

,dpm,dmg,kpar,dpar,gd15,csd15,xpd15
0,448.99,14919.87,0.6,0.24,60.35,7.57,60.49


Ruler


,callName,dpm,dmg,kpar,dpar,gd15,csd15,xpd15
9,Ruler,505.0,16664.62,0.58,0.27,548.38,24.66,389.0


Ghost


,callName,dpm,dmg,kpar,dpar,gd15,csd15,xpd15
3,Ghost,469.33,15674.33,0.65,0.22,38.0,3.35,152.83


In [157]:
pd.DataFrame( ad_15mean.mean()).T[['gd15','csd15','xpd15']]


,gd15,csd15,xpd15
0,-27.999306,-1.535985,-25.499937


In [174]:
# 세나를 사용한 원딜러들의 승률 확인
ad_carry[ad_carry['Champion'] =='세나'][['callName','games','winRate']].drop(index=5936)

,callName,games,winRate
6585,Teddy,5,0.80
5071,Bang,1,0.00
6071,Ruler,1,1.00
6800,Deft,4,0.75
12022,Noah,2,0.50


In [166]:
# 고스트의 세나, 킬관여율과 데미지 관여율 확인
Ghost_data[Ghost_data['Champion'] =='세나'][['kpar','dpar']]

,kpar,dpar
5936,0.7777,0.2295


In [182]:
# 세나를 가장 잘쓰는 팀을 확인하려고 함
display(qwer_data3[qwer_data3['Champion']== '세나'][['callName','team','Champion','games','winRate']])
display(qwer_data3[qwer_data3['Champion']== '세나'][['callName','team','Champion','winRate']].groupby(by='team').mean())

,callName,team,Champion,games,winRate
5071,Bang,Afreeca Freecs,세나,1,0.00
5936,Ghost,DWG KIA,세나,7,1.00
6071,Ruler,Gen.G,세나,1,1.00
6585,Teddy,T1,세나,5,0.80
6800,Deft,Hanwha Life Esports,세나,4,0.75
8749,HyBriD,KT Rolster,세나,1,0.00
11386,Becca,DRX,세나,1,0.00
11754,Vsta,Hanwha Life Esports,세나,1,0.00
12022,Noah,KT Rolster,세나,2,0.50


,winRate
team,
Afreeca Freecs,0.000
DRX,0.000
DWG KIA,1.000
Gen.G,1.000
Hanwha Life Esports,0.375
KT Rolster,0.250
T1,0.800


In [193]:
# 세나를 플레이한 게임 수 확인
qwer_data3[qwer_data3['Champion']== '세나'][['callName','team','Champion','games']].groupby(by='team').sum().sort_values('games')

,games
team,
Afreeca Freecs,1
DRX,1
Gen.G,1
KT Rolster,3
Hanwha Life Esports,5
T1,5
DWG KIA,7


In [188]:
# 전체 세나의 승률은 48%정도

qwer_data3[qwer_data3['Champion']== '세나'][['callName','team','Champion','winRate']].groupby(by='team').mean().mean()

winRate    0.489286
dtype: float64

In [199]:
# 팀원들이 보기 편하게 한 표로 정리하려고 함
aa = qwer_data3[qwer_data3['Champion']== '세나'][['callName','team','Champion','winRate']].groupby(by='team').mean().sort_values('winRate')
bb = qwer_data3[qwer_data3['Champion']== '세나'][['callName','team','Champion','games']].groupby(by='team').sum().sort_values('games',ascending=False)

In [205]:
pd.concat([bb,aa], axis=1, names='team')

,games,winRate
team,,
DWG KIA,7,1.000
Hanwha Life Esports,5,0.375
T1,5,0.800
KT Rolster,3,0.250
Afreeca Freecs,1,0.000
DRX,1,0.000
Gen.G,1,1.000


In [210]:
# 세나가 사용되었을 때 세나의 스탯을 확인
sena_stats = qwer_data3[qwer_data3['Champion']== '세나'].groupby('team').mean()
sena_stats

,games,wins,winRate,kills,deaths,assists,visionScore,wpm,wkm,dmg,...,cs15,gold15,ks,base,gd15,csd15,xpd15,lvd15,loses,kda
team,,,,,,,,,,,,,,,,,,,,,
Afreeca Freecs,1.0,0.0,0.000,0.000,3.000,3.000,91.000,1.730,0.440,6247.0,...,26.0,4133.0,0.00000,12.00,-2322.0,-131.00,-1159.0,-2.0,1.0,1.000
DRX,1.0,0.0,0.000,3.000,3.000,8.000,84.000,1.290,0.380,15542.0,...,15.0,3644.0,0.21420,15.00,394.0,-10.00,781.0,1.0,1.0,3.660
DWG KIA,7.0,7.0,1.000,4.570,1.000,9.280,84.850,1.350,0.510,17274.0,...,15.0,4515.0,0.25170,10.71,-435.0,-110.30,-911.0,-0.8,0.0,13.850
Gen.G,1.0,1.0,1.000,5.000,2.000,11.000,153.000,1.580,0.810,20893.0,...,14.0,4233.0,0.26310,16.00,-558.0,-111.00,-1171.0,-1.0,0.0,8.000
Hanwha Life Esports,2.5,1.5,0.375,2.625,2.875,6.625,79.875,1.265,0.470,17246.0,...,44.0,4190.0,0.19945,10.50,-286.0,-20.15,-218.0,-0.4,1.0,8.365
KT Rolster,1.5,0.5,0.250,0.750,1.750,4.000,69.500,0.880,0.455,13645.5,...,63.5,4402.0,0.07415,12.25,-1793.0,-71.75,-933.0,-1.0,1.0,3.080
T1,5.0,4.0,0.800,4.600,1.800,9.200,103.600,1.570,0.640,20383.0,...,24.0,4576.0,0.29460,12.80,-1090.0,-113.20,-969.0,-0.8,1.0,7.660


In [226]:
# 고스트 데이터를 다시 보여줘야하니까 잠시 가져옴
ghost = qwer_data3[qwer_data3['callName']== 'Ghost'][Ghost_data['games'] != 1].drop(columns= ['tournament','team'])

In [286]:
# 불필요한건 제거하고 필요한 값만
ghost[['callName','Champion','games','winRate','kills','deaths','assists','dmg','dpm','kpar','dpar','kda']]

,callName,Champion,games,winRate,kills,deaths,assists,dmg,dpm,kpar,dpar,kda
5936,Ghost,세나,7,1.0000,4.57,1.00,9.28,17274,572,0.7777,0.2295,13.85
5937,Ghost,트리스타나,3,1.0000,4.33,1.00,4.66,13523,468,0.6414,0.2346,9.00
5938,Ghost,카이사,20,0.8000,5.05,1.65,4.75,16662,476,0.6250,0.2173,5.93
5941,Ghost,칼리스타,3,0.6666,5.33,2.33,5.33,15178,417,0.6239,0.1700,4.57
5942,Ghost,아펠리오스,3,0.3333,1.66,2.00,3.00,12718,354,0.6032,0.2130,2.33
5943,Ghost,이즈리얼,3,0.6666,4.66,2.66,4.00,17781,576,0.7393,0.2637,3.25
5944,Ghost,자야,7,0.8571,5.42,2.28,5.57,18184,525,0.6474,0.2409,4.81


In [290]:
# 선수들 이름 확인
qwer_data3.callName.unique()

array(['Flawless', 'Bdd', 'Fly', 'Zzus', 'Peanut', 'Faker', 'Blank',
       'Bang', 'Ghost', 'Ruler', 'Lehends', 'Teddy', 'UmTi', 'Deft',
       'Kiin', 'Cuzz', 'Rascal', 'Khan', 'Lava', 'Yaharong', 'Ucal',
       'Effort', 'HyBriD', 'Prince', 'Chovy', 'Kingen', 'Leo',
       'ShowMaker', 'OnFleek', 'Dove', 'BeryL', 'Canyon', 'Summit',
       'Dread', 'Life', 'Kellin', 'Route', 'Clid', 'Rich', 'Doran',
       'FATE', 'Keria', 'deokdam', 'MapSSi', 'SOLKA', 'Pyosik', 'Becca',
       'BAO', 'Delight', 'Hena', 'Hoya', 'DuDu', 'Chieftain', 'Karis',
       'Canna', 'Clozer', 'Vsta', 'Ellim', 'Arthur', 'yoHan', 'Morgan',
       'GIDEON', 'Bonnie', 'Noah', 'Croco', 'Bay', 'Oner', 'Zeus',
       'Gumayusi'], dtype=object)

In [225]:
# 칸 선수의 데이터 확인용 
Khan = qwer_data3[qwer_data3['callName']== 'Khan'][qwer_data3['games'] != 1].drop(columns= ['tournament','team'])

<ipython-input-225-4e06d29dc5c8>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  Khan = qwer_data3[qwer_data3['callName']== 'Khan'][qwer_data3['games'] != 1].drop(columns= ['tournament','team'])


In [296]:
# 두 팀의 미드 선수의 데이터 확인용 
ShowMaker = qwer_data3[qwer_data3['callName']== 'ShowMaker'][qwer_data3['games'] != 1].drop(columns= ['tournament','team'])
BDD = qwer_data3[qwer_data3['callName']== 'Bdd'][qwer_data3['games'] != 1].drop(columns= ['tournament','team'])

<ipython-input-296-33549d7ce0cc>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ShowMaker = qwer_data3[qwer_data3['callName']== 'ShowMaker'][qwer_data3['games'] != 1].drop(columns= ['tournament','team'])
<ipython-input-296-33549d7ce0cc>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  BDD = qwer_data3[qwer_data3['callName']== 'Bdd'][qwer_data3['games'] != 1].drop(columns= ['tournament','team'])


In [294]:
ShowMaker[['callName','Champion','games','winRate','kills','deaths','assists','dmg','dpm','kpar','dpar']]

,callName,Champion,games,winRate,kills,deaths,assists,dmg,dpm,kpar,dpar
9600,ShowMaker,빅토르,9,0.8888,3.22,1.55,5.22,17331,587,0.6745,0.2950
9601,ShowMaker,질리언,2,1.0000,2.00,0.00,14.00,17392,501,0.8000,0.2044
9602,ShowMaker,조이,9,0.7777,5.00,0.88,7.22,20628,641,0.6749,0.2728
9603,ShowMaker,세라핀,3,1.0000,3.33,1.00,9.00,12668,415,0.8481,0.1884
9605,ShowMaker,아지르,4,1.0000,3.00,1.50,8.25,16475,513,0.6155,0.2193
9606,ShowMaker,루시안,3,1.0000,6.00,1.33,6.00,20024,662,0.7419,0.2622
9607,ShowMaker,신드라,12,0.7500,3.33,2.00,6.25,21596,582,0.6262,0.2571
9610,ShowMaker,오리아나,2,0.0000,1.00,3.00,4.50,13112,304,0.4772,0.1733


In [298]:
BDD[['callName','Champion','games','winRate','kills','deaths','assists','dmg','dpm','kpar','dpar','gd15','csd15','xpd15']]

,callName,Champion,games,winRate,kills,deaths,assists,dmg,dpm,kpar,dpar,gd15,csd15,xpd15
1662,Bdd,아지르,14,0.7142,2.35,1.78,6.14,18626,574,0.6624,0.2935,119,8.2,357
1664,Bdd,오리아나,9,0.6666,2.33,1.44,5.22,12363,404,0.6572,0.2617,661,10.4,444
1665,Bdd,조이,4,0.5000,3.25,1.25,7.25,27445,670,0.6906,0.3812,219,7.7,23
1666,Bdd,신드라,19,0.6315,3.52,1.68,4.15,15515,485,0.6364,0.2791,-30,2.5,-37


In [245]:
# 고스트랑 비교할 리그 원딜러들의 평균 확인
ad_carry2.groupby('Champion').mean().mean()

games                2.860542
wins                 1.322441
winRate              0.327509
kills                2.998811
deaths               2.674383
assists              4.053132
visionScore         43.665209
wpm                  0.520629
wkm                  0.366099
dmg              15263.148071
dpm                461.762993
dtpm               448.335090
kapm                 0.214137
gold             13571.411523
gpm                410.696359
cs                 304.337930
cspm                 9.225009
counterJungle        1.529756
kpar                 0.615019
dpar                 0.247834
xp15              5645.198631
cs15               135.472995
gold15            5469.655227
ks                   0.253466
base                11.666790
gd15               212.959002
csd15               12.070723
xpd15              170.223727
lvd15                0.092005
loses                1.538102
kda                  3.908784
dtype: float64

In [285]:
ghost[ghost['Champion'] == '세나'][['callName','dpm','dmg','kpar','dpar','gd15','csd15','xpd15','kda']].round(2)

,callName,dpm,dmg,kpar,dpar,gd15,csd15,xpd15,kda
5936,Ghost,572,17274,0.78,0.23,-435,-110.3,-911,13.85


In [7]:
qwer_data3

,callName,tournament,team,games,wins,winRate,kills,deaths,assists,visionScore,...,gold15,ks,base,gd15,csd15,xpd15,lvd15,loses,kda,Champion
140,Flawless,LCK 2021 Spring Season,Gen.G,1,1,1.0000,2.00,3.00,12.00,52.00,...,4651,0.0869,12.00,-324,-1.0,-422,0.0,0,4.66,판테온
141,Flawless,LCK 2021 Spring Season,Gen.G,2,1,0.5000,4.00,2.00,3.00,44.00,...,4875,0.4444,9.50,44,4.0,562,1.0,1,3.50,올라프
142,Flawless,LCK 2021 Spring Season,Gen.G,1,1,1.0000,2.00,1.00,5.00,91.00,...,5148,0.1428,19.00,-58,-21.0,-1002,-1.0,0,7.00,릴리아
143,Flawless,LCK 2021 Spring Season,Gen.G,1,0,0.0000,0.00,0.00,0.00,69.00,...,4500,0.0000,11.00,-1828,-26.0,-1825,-1.0,1,0.00,우디르
1662,Bdd,LCK 2021 Spring Season,Gen.G,14,10,0.7142,2.35,1.78,6.14,42.57,...,5491,0.1646,11.57,119,8.2,357,0.2,4,4.76,아지르
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12071,Gumayusi,LCK 2021 Spring Season,T1,3,2,0.6666,4.33,2.33,4.33,52.66,...,5481,0.3493,12.33,-520,-7.4,-224,-0.4,1,3.71,베인
12072,Gumayusi,LCK 2021 Spring Season,T1,4,2,0.5000,7.75,2.25,6.00,55.75,...,6214,0.4928,12.25,1305,24.5,615,0.5,2,6.11,사미라
12073,Gumayusi,LCK 2021 Spring Season,T1,6,2,0.3333,2.50,2.00,2.66,46.33,...,5420,0.3326,12.50,-109,5.1,172,0.1,4,2.58,카이사
12074,Gumayusi,LCK 2021 Spring Season,T1,2,0,0.0000,3.50,2.50,1.00,29.00,...,5206,0.4375,10.50,-231,1.5,-219,0.0,2,1.80,진


In [23]:
# 모든 라이너들을 확인하기 위해 팀으로 구분

a = qwer_data3[qwer_data3['team'] == 'DWG KIA']
b = qwer_data3[qwer_data3['team'] == 'Gen.G']

In [20]:
# 담원 선수들의 각 기록 평균
a.groupby('callName').mean()[['dpm','dmg','kpar','dpar','gd15','csd15','xpd15','kda']].round(2)

,dpm,dmg,kpar,dpar,gd15,csd15,xpd15,kda
callName,,,,,,,,
BeryL,230.53,7783.27,0.66,0.10,378.13,18.83,319.40,4.35
Canyon,414.22,14051.11,0.70,0.20,350.33,8.08,489.78,5.67
Ghost,498.50,16488.40,0.67,0.23,-146.60,-10.62,-82.50,7.26
Khan,483.21,16436.57,0.61,0.23,48.79,-3.36,-134.57,3.54
ShowMaker,492.54,16932.08,0.73,0.23,493.08,5.86,266.54,9.01


### 칸의 15분 격차는 사이온을 많이 써서 그런거라고 생각되고


In [25]:
# 젠지 선수들의 각 기록 평균
b.groupby('callName').mean()[['dpm','dmg','kpar','dpar','gd15','csd15','xpd15','kda']].round(2)

,dpm,dmg,kpar,dpar,gd15,csd15,xpd15,kda
callName,,,,,,,,
Bdd,499.20,17427.60,0.63,0.28,141.60,1.76,112.40,6.59
Clid,290.71,9606.00,0.66,0.16,67.00,-0.84,24.71,4.85
Flawless,252.50,8880.25,0.43,0.13,-541.50,-11.00,-671.75,3.79
Karis,274.00,9200.00,0.37,0.19,-549.50,-14.00,-556.50,1.50
Life,141.17,4813.83,0.53,0.07,-143.25,-11.28,-244.50,4.18
Rascal,391.69,13381.08,0.60,0.24,431.85,10.48,300.31,4.44
Ruler,459.91,15564.73,0.57,0.25,370.00,8.12,95.91,8.28


In [44]:
b[b['callName']== 'Rascal']

,callName,tournament,team,games,wins,winRate,kills,deaths,assists,visionScore,...,gold15,ks,base,gd15,csd15,xpd15,lvd15,loses,kda,Champion
7729,Rascal,LCK 2021 Spring Season,Gen.G,9,7,0.7777,3.00,1.66,4.77,30.33,...,5859,0.2559,9.77,685,14.6,303,0.3,2,4.66,레넥톤
7730,Rascal,LCK 2021 Spring Season,Gen.G,4,2,0.5000,1.00,2.50,4.25,28.00,...,4802,0.1067,10.25,-866,-16.5,-260,0.0,2,2.10,사이온
7731,Rascal,LCK 2021 Spring Season,Gen.G,6,4,0.6666,0.83,1.33,5.33,30.00,...,5029,0.0592,14.50,-32,2.3,56,0.0,2,4.62,오른
7732,Rascal,LCK 2021 Spring Season,Gen.G,4,2,0.5000,2.50,4.00,6.50,29.50,...,5809,0.2528,10.25,837,6.5,499,0.2,2,2.25,나르
7733,Rascal,LCK 2021 Spring Season,Gen.G,5,2,0.4000,2.40,3.60,5.20,46.00,...,4786,0.1770,15.20,-887,-14.6,-473,-0.2,3,2.11,볼리베어
7734,Rascal,LCK 2021 Spring Season,Gen.G,1,1,1.0000,5.00,1.00,6.00,29.00,...,5917,0.3333,11.00,1531,39.0,957,1.0,0,11.00,판테온
7735,Rascal,LCK 2021 Spring Season,Gen.G,2,2,1.0000,3.00,1.00,6.00,32.00,...,5019,0.1944,9.50,405,9.5,467,0.0,0,9.00,아칼리
7736,Rascal,LCK 2021 Spring Season,Gen.G,11,7,0.6363,3.09,3.00,4.36,35.36,...,5317,0.1903,12.00,-77,-5.2,-136,0.0,4,2.48,카밀
7737,Rascal,LCK 2021 Spring Season,Gen.G,1,1,1.0000,5.00,5.00,2.00,59.00,...,6111,0.3571,20.00,1312,28.0,402,0.0,0,1.40,퀸
7738,Rascal,LCK 2021 Spring Season,Gen.G,4,3,0.7500,2.50,1.50,5.25,37.75,...,5114,0.2105,9.75,144,9.7,368,0.7,1,5.16,초가스


In [33]:
qwer_data3['callName'].unique()

array(['Flawless', 'Bdd', 'Fly', 'Zzus', 'Peanut', 'Faker', 'Blank',
       'Bang', 'Ghost', 'Ruler', 'Lehends', 'Teddy', 'UmTi', 'Deft',
       'Kiin', 'Cuzz', 'Rascal', 'Khan', 'Lava', 'Yaharong', 'Ucal',
       'Effort', 'HyBriD', 'Prince', 'Chovy', 'Kingen', 'Leo',
       'ShowMaker', 'OnFleek', 'Dove', 'BeryL', 'Canyon', 'Summit',
       'Dread', 'Life', 'Kellin', 'Route', 'Clid', 'Rich', 'Doran',
       'FATE', 'Keria', 'deokdam', 'MapSSi', 'SOLKA', 'Pyosik', 'Becca',
       'BAO', 'Delight', 'Hena', 'Hoya', 'DuDu', 'Chieftain', 'Karis',
       'Canna', 'Clozer', 'Vsta', 'Ellim', 'Arthur', 'yoHan', 'Morgan',
       'GIDEON', 'Bonnie', 'Noah', 'Croco', 'Bay', 'Oner', 'Zeus',
       'Gumayusi'], dtype=object)

## 다른 정글러들 스탯 확인

In [34]:
qwer_data3[qwer_data3['callName'] == 'Ellim'].groupby('callName').mean()[['dpm','dmg','kpar','dpar','gd15','csd15','xpd15','kda']].round(2)

,dpm,dmg,kpar,dpar,gd15,csd15,xpd15,kda
callName,,,,,,,,
Ellim,263.14,8285.29,0.69,0.15,88.71,4.54,33.29,6.71


In [36]:
qwer_data3[qwer_data3['callName'] == 'Cuzz'].groupby('callName').mean()[['dpm','dmg','kpar','dpar','gd15','csd15','xpd15','kda']].round(2)

,dpm,dmg,kpar,dpar,gd15,csd15,xpd15,kda
callName,,,,,,,,
Cuzz,331.88,11308.5,0.61,0.16,-78.5,-3.09,-99.38,9.02


In [37]:
qwer_data3[qwer_data3['callName'] == 'Arthur'].groupby('callName').mean()[['dpm','dmg','kpar','dpar','gd15','csd15','xpd15','kda']].round(2)

,dpm,dmg,kpar,dpar,gd15,csd15,xpd15,kda
callName,,,,,,,,
Arthur,310.44,9907.56,0.64,0.15,-165.33,-7.32,-436.0,3.6


In [39]:
qwer_data3[qwer_data3['callName'] == 'Clid'].groupby('callName').mean()[['dpm','dmg','kpar','dpar','gd15','csd15','xpd15','kda']].round(2)

,dpm,dmg,kpar,dpar,gd15,csd15,xpd15,kda
callName,,,,,,,,
Clid,290.71,9606.0,0.66,0.16,67.0,-0.84,24.71,4.85


In [40]:
qwer_data3[qwer_data3['callName'] == 'Pyosik'].groupby('callName').mean()[['dpm','dmg','kpar','dpar','gd15','csd15','xpd15','kda']].round(2)

,dpm,dmg,kpar,dpar,gd15,csd15,xpd15,kda
callName,,,,,,,,
Pyosik,335.09,11333.64,0.69,0.18,-288.0,-1.0,-188.18,2.81


In [41]:
qwer_data3[qwer_data3['callName'] == 'Canyon'].groupby('callName').mean()[['dpm','dmg','kpar','dpar','gd15','csd15','xpd15','kda']].round(2)

,dpm,dmg,kpar,dpar,gd15,csd15,xpd15,kda
callName,,,,,,,,
Canyon,414.22,14051.11,0.7,0.2,350.33,8.08,489.78,5.67


In [46]:
a[a['callName'] == 'Ghost'].sort_values('games', ascending = False)

,callName,tournament,team,games,wins,winRate,kills,deaths,assists,visionScore,...,gold15,ks,base,gd15,csd15,xpd15,lvd15,loses,kda,Champion
5938,Ghost,LCK 2021 Spring Season,DWG KIA,20,16,0.8000,5.05,1.65,4.75,60.40,...,5309,0.3163,11.80,-184,-6.0,-162,-0.2,4,5.93,카이사
5936,Ghost,LCK 2021 Spring Season,DWG KIA,7,7,1.0000,4.57,1.00,9.28,84.85,...,4515,0.2517,10.71,-435,-110.3,-911,-0.8,0,13.85,세나
5944,Ghost,LCK 2021 Spring Season,DWG KIA,7,6,0.8571,5.42,2.28,5.57,41.71,...,5395,0.3271,11.42,150,-8.5,157,0.1,1,4.81,자야
5937,Ghost,LCK 2021 Spring Season,DWG KIA,3,3,1.0000,4.33,1.00,4.66,42.33,...,5512,0.2996,9.00,243,-1.7,311,0.3,0,9.00,트리스타나
5941,Ghost,LCK 2021 Spring Season,DWG KIA,3,2,0.6666,5.33,2.33,5.33,72.66,...,5730,0.2970,13.33,410,9.0,530,0.0,1,4.57,칼리스타
5942,Ghost,LCK 2021 Spring Season,DWG KIA,3,1,0.3333,1.66,2.00,3.00,46.00,...,5501,0.2262,13.00,94,-0.7,-139,0.0,2,2.33,아펠리오스
5943,Ghost,LCK 2021 Spring Season,DWG KIA,3,2,0.6666,4.66,2.66,4.00,44.66,...,5083,0.3892,10.00,-485,28.0,220,0.0,1,3.25,이즈리얼
5939,Ghost,LCK 2021 Spring Season,DWG KIA,1,1,1.0000,4.00,1.00,10.00,38.00,...,5082,0.2105,12.00,-1161,-12.0,-341,0.0,0,14.00,바루스
5940,Ghost,LCK 2021 Spring Season,DWG KIA,1,1,1.0000,8.00,0.00,4.00,51.00,...,5325,0.6153,10.00,118,8.0,-326,-1.0,0,14.40,징크스
5945,Ghost,LCK 2021 Spring Season,DWG KIA,1,0,0.0000,0.00,2.00,1.00,41.00,...,5311,0.0000,13.00,-216,-12.0,-164,0.0,1,0.50,사미라


In [50]:
a[a['callName'] == 'BeryL'].sort_values('games', ascending = False)

,callName,tournament,team,games,wins,winRate,kills,deaths,assists,visionScore,...,gold15,ks,base,gd15,csd15,xpd15,lvd15,loses,kda,Champion
9893,BeryL,LCK 2021 Spring Season,DWG KIA,9,7,0.7777,1.66,4.22,9.00,90.55,...,3560,0.0866,14.22,189,3.4,414,0.5,2,2.52,그라가스
9897,BeryL,LCK 2021 Spring Season,DWG KIA,8,6,0.7500,0.87,3.62,7.37,102.75,...,3314,0.0841,15.00,108,11.0,145,0.2,2,2.27,알리스타
9887,BeryL,LCK 2021 Spring Season,DWG KIA,7,7,1.0000,1.42,2.42,9.14,86.28,...,3607,0.0983,12.42,418,3.1,288,0.2,0,4.35,렐
9889,BeryL,LCK 2021 Spring Season,DWG KIA,4,4,1.0000,0.75,0.75,10.75,39.00,...,4605,0.0392,10.00,1533,72.7,1084,1.2,0,15.33,탐 켄치
9895,BeryL,LCK 2021 Spring Season,DWG KIA,4,3,0.7500,0.75,2.75,12.50,80.50,...,3614,0.0433,12.25,18,-24.0,-410,-0.5,1,4.81,세라핀
9886,BeryL,LCK 2021 Spring Season,DWG KIA,3,1,0.3333,1.00,5.66,8.33,106.66,...,3621,0.1022,15.66,154,7.0,266,0.3,2,1.64,레오나
9890,BeryL,LCK 2021 Spring Season,DWG KIA,3,1,0.3333,1.00,3.66,7.66,89.33,...,3584,0.1268,14.33,414,2.3,366,0.0,2,2.36,갈리오
9884,BeryL,LCK 2021 Spring Season,DWG KIA,2,2,1.0000,1.50,1.00,7.50,50.00,...,4412,0.0833,9.50,1098,51.0,612,1.0,0,9.00,마오카이
9888,BeryL,LCK 2021 Spring Season,DWG KIA,2,2,1.0000,0.50,3.50,8.50,113.00,...,3405,0.0312,14.50,142,-17.5,-437,-0.5,0,2.57,라칸
9894,BeryL,LCK 2021 Spring Season,DWG KIA,2,2,1.0000,4.00,3.50,6.00,34.50,...,5155,0.2916,12.50,1757,95.5,2089,2.0,0,2.85,하이머딩거


In [49]:
qwer_data3[qwer_data3['team'] == 'Gen.G'].groupby('team').mean()

,games,wins,winRate,kills,deaths,assists,visionScore,wpm,wkm,dmg,...,cs15,gold15,ks,base,gd15,csd15,xpd15,lvd15,loses,kda
team,,,,,,,,,,,,,,,,,,,,,
Gen.G,4.62963,2.962963,0.607741,2.171296,1.996667,5.18537,58.932778,0.798704,0.384259,11319.12963,...,103.444444,4890.111111,0.169763,12.32537,108.833333,0.390741,-19.259259,0.042593,1.666667,5.262778


# 크롤링 데이터 활용하기

In [42]:
# gol.gg 데이터를 확인

data = pd.read_csv('DKGAMEDATA.csv', index_col=0)
data

,gametime,blue_teamname,blue_result,red_teamname,red_result,client_version,call_name,champion,kill,death,...,wcpm,vs_per,Total_damage,dpm,dmg_per,gd15,csd15,xpd15,lvld15,to_dmg_turrets
0,22:26,DWG KIA,WIN,KT Rolster,LOSS,v11.5,Khan,Gangplank,4,1,...,0.27,0.09,14.6%,719,29.3%,982,-1,884,1,5666
1,22:26,DWG KIA,WIN,KT Rolster,LOSS,v11.5,Canyon,Nidalee,5,0,...,0.18,0.36,20.4%,401,16.4%,796,30,788,0,1271
2,22:26,DWG KIA,WIN,KT Rolster,LOSS,v11.5,ShowMaker,Azir,1,1,...,0.18,0.22,18.2%,349,14.3%,-343,-13,-464,0,4207
3,22:26,DWG KIA,WIN,KT Rolster,LOSS,v11.5,BeryL,Tahm Kench,0,0,...,0.27,0.04,9.5%,282,11.5%,1685,28,219,1,2483
4,22:26,DWG KIA,WIN,KT Rolster,LOSS,v11.5,Ghost,Senna,5,1,...,0.13,0.31,37.2%,699,28.5%,2795,-8,920,1,5994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,33:22,T1,LOSS,DWG KIA,WIN,v11.1,Khan,Gangplank,7,1,...,0.21,0.18,13.2%,646,28.7%,1780,23,382,0,9921
416,33:22,T1,LOSS,DWG KIA,WIN,v11.1,Canyon,Graves,2,0,...,0.33,0.69,21.4%,429,19.1%,415,10,386,0,2075
417,33:22,T1,LOSS,DWG KIA,WIN,v11.1,ShowMaker,Zoe,5,0,...,0.18,0.27,14.1%,588,26.2%,709,-5,-247,0,5264
418,33:22,T1,LOSS,DWG KIA,WIN,v11.1,Ghost,Kai,4,2,...,0.30,0.24,14.8%,437,19.5%,-879,-30,-292,0,4913


In [44]:
# 담원의 세나가 활용된 매치업 확인
Senna_data = data[data['champion'] == 'Senna']
Senna_data

,gametime,blue_teamname,blue_result,red_teamname,red_result,client_version,call_name,champion,kill,death,...,wcpm,vs_per,Total_damage,dpm,dmg_per,gd15,csd15,xpd15,lvld15,to_dmg_turrets
4,22:26,DWG KIA,WIN,KT Rolster,LOSS,v11.5,Ghost,Senna,5,1,...,0.13,0.31,37.2%,699,28.5%,2795,-8,920,1,5994
129,28:09,Afreeca Freecs,LOSS,DWG KIA,WIN,v11.4,Ghost,Senna,6,1,...,0.21,0.36,38.9%,628,19%,729,-13,233,0,6513
159,31:35,DWG KIA,LOSS,T1,WIN,v11.3,Teddy,Senna,3,2,...,0.32,0.57,32.8%,584,23%,934,12,394,1,2784
169,28:38,T1,LOSS,DWG KIA,WIN,v11.3,Ghost,Senna,5,1,...,0.35,0.42,33.7%,368,18.6%,2143,13,215,1,4966
199,34:42,Hanwha Life eSports,LOSS,DWG KIA,WIN,v11.3,Ghost,Senna,10,2,...,0.23,0.55,35.9%,938,31.2%,807,-2,1153,1,7011
294,29:42,Hanwha Life eSports,LOSS,DWG KIA,WIN,v11.2,Vsta,Senna,1,5,...,0.67,0.40,35.6%,341,16.2%,-703,5,-137,0,0
308,34:44,DWG KIA,WIN,Hanwha Life eSports,LOSS,v11.2,Deft,Senna,3,0,...,0.14,0.60,19.9%,567,35.3%,408,17,-160,0,1274


In [45]:
# 게임을 구분할 방법이 게임타임이라 게임타임으로 담원이 세나한테 졌던 경기 확인
data[data['gametime'] == '31:35']

,gametime,blue_teamname,blue_result,red_teamname,red_result,client_version,call_name,champion,kill,death,...,wcpm,vs_per,Total_damage,dpm,dmg_per,gd15,csd15,xpd15,lvld15,to_dmg_turrets
150,31:35,DWG KIA,LOSS,T1,WIN,v11.3,Khan,Sion,2,4,...,0.16,0.03,8%,495,25.4%,96,-7,222,1,0
151,31:35,DWG KIA,LOSS,T1,WIN,v11.3,Canyon,Kayn,4,5,...,0.32,0.60,30.1%,431,22.1%,-2708,-33,-1524,-1,169
152,31:35,DWG KIA,LOSS,T1,WIN,v11.3,ShowMaker,Syndra,1,4,...,0.06,0.13,9.6%,292,15%,-554,-3,-504,-1,1701
153,31:35,DWG KIA,LOSS,T1,WIN,v11.3,Ghost,Ezreal,2,4,...,0.25,0.44,19.7%,525,26.9%,955,35,150,0,2924
154,31:35,DWG KIA,LOSS,T1,WIN,v11.3,BeryL,Seraphine,0,7,...,0.47,0.32,32.5%,208,10.7%,-934,-12,-394,-1,330
155,31:35,DWG KIA,LOSS,T1,WIN,v11.3,Zeus,Camille,3,3,...,0.09,0.19,11.8%,452,17.8%,-96,7,-222,-1,5595
156,31:35,DWG KIA,LOSS,T1,WIN,v11.3,Oner,Lillia,8,0,...,0.19,0.60,21.4%,585,23.1%,2708,33,1524,1,2099
157,31:35,DWG KIA,LOSS,T1,WIN,v11.3,Clozer,Lucian,8,3,...,0.13,0.19,20.6%,630,24.8%,554,3,504,1,5120
158,31:35,DWG KIA,LOSS,T1,WIN,v11.3,Keria,Nautilus,2,1,...,0.35,0.16,13.4%,285,11.2%,-955,-35,-150,0,401
159,31:35,DWG KIA,LOSS,T1,WIN,v11.3,Teddy,Senna,3,2,...,0.32,0.57,32.8%,584,23%,934,12,394,1,2784


## 위에서 확인했던 내용 다시 재 확인

In [79]:
a = qwer_data3[qwer_data3['team'] == 'Gen.G']

In [83]:
a[a['callName'] == 'Bdd'].groupby('callName').mean()

,games,wins,winRate,kills,deaths,assists,visionScore,wpm,wkm,dmg,...,cs15,gold15,ks,base,gd15,csd15,xpd15,lvd15,loses,kda
callName,,,,,,,,,,,,,,,,,,,,,
Bdd,9.4,6.2,0.70246,2.69,1.43,6.152,38.598,0.486,0.26,17427.6,...,148.0,5606.8,0.21474,12.432,141.6,1.76,112.4,0.06,3.2,6.59


In [87]:
b = qwer_data3[qwer_data3['callName'] == 'Ghost']

In [94]:
c = b[b['Champion'] == '세나']
d = b[b['Champion'] != '세나']

In [93]:
c[['callName','Champion','games','wins','winRate','kda','kpar']]

,callName,Champion,games,wins,winRate,kda,kpar
5936,Ghost,세나,7,7,1.0,13.85,0.7777


In [100]:
e  = d[['callName','Champion','games','wins','winRate','kda','kpar']]

In [106]:
e[e['games'] != 1]

,callName,Champion,games,wins,winRate,kda,kpar
5937,Ghost,트리스타나,3,3,1.0000,9.00,0.6414
5938,Ghost,카이사,20,16,0.8000,5.93,0.6250
5941,Ghost,칼리스타,3,2,0.6666,4.57,0.6239
5942,Ghost,아펠리오스,3,1,0.3333,2.33,0.6032
5943,Ghost,이즈리얼,3,2,0.6666,3.25,0.7393
5944,Ghost,자야,7,6,0.8571,4.81,0.6474
